In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import keras.models as models
import keras.layers as layers
from keras import backend

In [ ]:
! unzip /content/drive/MyDrive/planet-understanding-the-amazon-from-space.zip

In [ ]:
! unzip /content/drive/MyDrive/planet-mini.zip

In [ ]:
train_label =pd.read_csv('/content/train_v2.csv/train_v2.csv')

In [ ]:
train_label.head()

In [ ]:
train_label.shape

In [ ]:
Path ='/content/planet/planet/train-jpg'

In [ ]:
import cv2

def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz,1);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

# CUSTOM
def cv2_to_plt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def dehaze(image_path:Path, output_path:Path):
    src = cv2.imread(str(image_path))

    I = src.astype('float64')/255
    dark = DarkChannel(I,15)
    A = AtmLight(I,dark)
    te = TransmissionEstimate(I,A,15)
    t = TransmissionRefine(src,te)
    J = Recover(I,t,A,0.1)
    
    cv2.imwrite(str(output_path/image_path.name), J*255);

In [ ]:
import math
fn = '/content/planet/planet/train-jpg/train_10.jpg'

src = cv2.imread(fn)

I = src.astype('float64')/255

dark = DarkChannel(I,15)
A = AtmLight(I,dark)
te = TransmissionEstimate(I,A,15)
t = TransmissionRefine(src,te)
J = Recover(I,t,A,0.1);

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(dark);

In [ ]:
plt.imshow(cv2_to_plt(src));

In [ ]:
plt.imshow(cv2_to_plt(J.astype('float32')))

In [ ]:
test_df=pd.read_csv('/content/test_v2_file_mapping.csv/test_v2_file_mapping.csv')

In [ ]:
test_df.head()

In [ ]:
def tag_mapping(data):
    labels=set()
    for i in range(len(data)):
        tags=data['tags'][i].split(' ')
        labels.update(tags)
    labels=list(labels)
    labels.sort()
    labels_dict={labels[i]:i for i in range(len(labels))}
    inv_labels={i:labels[i] for i in range(len(labels))}
    return labels_dict,inv_labels

In [ ]:
label_map,invmap=tag_mapping(train_label)

In [ ]:
def file_mapping(data):
    mapping={}
    for i in range(len(data)):
        name,tags=train_label['image_name'][i],train_label['tags'][i]
        mapping[name]=tags.split(' ')
    return mapping

In [ ]:
def one_hot_encode(tags, mapping):
    encoding = np.zeros(len(mapping), dtype='uint8')
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding

In [ ]:
def load_dataset(path,file_mapping,tag_mapping):
    photos,targets=list(),list()
    for filename in os.listdir(path):
        photo=load_img(path+filename,target_size=(75,75))
        photo=img_to_array(photo,dtype='uint8')
        tags=file_mapping[filename[:-4]]
        target=one_hot_encode(tags,tag_mapping)
        photos.append(photo)
        targets.append(target)
    X=np.asarray(photos,dtype='uint8')
    y=np.asarray(targets,dtype='uint8')
    return X,y

In [ ]:
tags_mapping,_=tag_mapping(train_label)
files_mapping=file_mapping(train_label)
path='/content/planet/planet/train-jpg/'
X,y=load_dataset(path,files_mapping,tags_mapping)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, RMSprop

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
def fbeta(y_true, y_pred, beta=2):
    y_pred = backend.clip(y_pred, 0, 1)

    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    p = tp / (tp + fp + backend.epsilon())
    r = tp / (tp + fn + backend.epsilon())
    bb = beta ** 2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score

In [ ]:
from tensorflow.keras.applications import InceptionV3
model=InceptionV3(input_shape=(75,75,3),include_top=False)
for layer in model.layers:
    layers.trainable=False
last_layer=model.get_layer('mixed7')
last_output=last_layer.output
    
x=layers.Flatten()(last_output)
x=layers.Dense(256,activation='relu')(x)
x=layers.Dense(128,activation='relu')(x)
x=layers.Dropout(0.2)(x)
x=layers.Dense(17,activation='sigmoid')(x)
model=models.Model(model.inputs,x)
model.compile(optimizer=RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=[fbeta])

In [ ]:
train_datagen=ImageDataGenerator(rescale=1.0/255.0)
test_datagen=ImageDataGenerator(rescale=1.0/255.0)
# ,horizontal_flip=False, vertical_flip=False, rotation_range=90

In [ ]:
# train_gen=train_datagen.flow(X_train,y_train,batch_size=64)
# test_gen=test_datagen.flow(X_test,y_test,batch_size=64)

In [ ]:
# history = model.fit(train_gen,steps_per_epoch=506,validation_data=test_gen, validation_steps=127, epochs=50, verbose=0)

In [ ]:
train_gen=train_datagen.flow(X_train,y_train,batch_size=32)
test_gen=test_datagen.flow(X_test,y_test,batch_size=32)

In [ ]:
history = model.fit(train_gen,steps_per_epoch=128,validation_data=test_gen, validation_steps=128, epochs=10, verbose=0)

In [ ]:
loss, fbeta =model.evaluate_generator(test_gen, steps=8, verbose=0)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
history.history['fbeta']

In [ ]:
test_data1='/content/planet/planet/test-jpg/'
test_data2='/content/test-jpg-additional/test-jpg-additional/'

In [ ]:
submission_df=pd.read_csv('/content/sample_submission_v2.csv/sample_submission_v2.csv')

In [ ]:
photo_test=[]
for filename in submission_df['image_name']:
    if filename[:1]=='t':
        img=load_img(test_data1+filename+'.jpg',target_size=(75,75))
    elif filename[:1]=='f':
        img=load_img(test_data2+filename+'.jpg',target_size=(75,75))
    ph=img_to_array(img,dtype='uint8')
    photo_test.append(ph)

In [ ]:
test_X=np.asarray(photo_test,dtype='uint8')

In [ ]:
image_gen_test=ImageDataGenerator(rescale=1/255.0)
test_data_gen=image_gen_test.flow(test_X,shuffle=False,batch_size=1024)

In [ ]:
prediction=model.predict(test_data_gen)

In [ ]:
answer=pd.DataFrame(prediction,columns=tags_mapping.keys())

In [ ]:
tags=answer.columns
pred_tags=answer.apply(lambda x: ' '.join(tags[x>0.5]),axis=1)

In [ ]:
pred_tag=pd.DataFrame(pred_tags,columns=['tags'])

In [ ]:
submission_df['tags']=pred_tag['tags']

In [ ]:
submission_df.to_csv('ayusuf_submission.csv',index=False)

In [ ]:
submission_df.head(10)